# ANALYSIS 5 : RECOMMENDER SYSTEM FOR BRAND RUBIE'S COSTUME CO.

Collaborative filtering algorithms is used to get the recomendations.
The Recommender System will take the 'Product Name' and based on the correlation factor will give output as list of products which will be a suggestion or recommendation.
Suppose product name 'A' act as input parameter i.e. If a user buy product 'A' so based on that it will output the product highly correlated to it.

In [40]:
import glob
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [63]:
files=glob.glob('E:\CapstoneProject\Prime_Pantry.json\*')

Cleaning of 'meta_Prime_Pantry.json' file and importing the data as pandas DataFrame.

In [82]:
# Reading a multiple json files from a single json file 'meta_Prime_Pantry.json'.
product=[]
with open(files[0]) as data_file:
    data=data_file.read()
    for i in data.split('\n'):
        product.append(i)
        
# Firstly cleaning the data by converting files into proper json format files by some replacements and 
# then Making a list of Tuples containg data of brand Rubie's Costume Co.
productDataframe=[]
brand_List=['Perrier']
for x in product:
    try:
        y=x.replace("'",'"')
        jdata=json.loads(y)
        if jdata['brand'] in brand_List:
            productDataframe.append((jdata['asin'],jdata['title'])) 
    except:
        pass
    
# Creating a dataframe using the list of Tuples got in the previous step.  
Product_dataset=pd.DataFrame(productDataframe,columns=['Asin','Title'])

Getting all the distinct product Asin in list.

In [83]:
prod_List=Product_dataset.Asin.unique().tolist()

In [84]:
prod_List

['B00VWH5C30',
 'B00VXIN01E',
 'B010S568YI',
 'B010S577L6',
 'B010S56PXC',
 'B011TZXD6W',
 'B01DLRDKPK',
 'B01DLRDD3Y']

Cleaning of 'Prime_Pantry.json' file and importing the data as pandas DataFrame.

In [85]:
# Reading a multiple json files from a single json file 'ReviewSample.json'.
review=[]
with open(files[1]) as data_file:
    data=data_file.read()
    for i in data.split('\n'):
        review.append(i)
        
# Making a list of Tuples containg all the data of json files.
reviewDataframe=[]
for x in review:
    try:
        jdata=json.loads(x)
        reviewDataframe.append((jdata['reviewerID'],jdata['asin'],jdata['overall'])) 
    except:
        pass
    
# Creating a dataframe using the list of Tuples got in the previous step.     
review_dataset=pd.DataFrame(reviewDataframe,columns=['Reviewer_ID','Asin','Rating'])

Creating a DataFrame 'Working_dataset' which has products only from brand "Perrier"

In [86]:
Working_dataset=review_dataset[review_dataset.Asin.isin(prod_List)]

In [87]:
Working_dataset

,Reviewer_ID,Asin,Rating
255529,A1WJORUDHLX1Z4,B00VWH5C30,5.0
255530,A1Z9L6HT5YWI8B,B00VWH5C30,4.0
255531,A3S9HOWOXT7LRC,B00VWH5C30,5.0
255532,A2ZDYGCO19YGEB,B00VWH5C30,5.0
255533,AHXGBJ6E749Y2,B00VWH5C30,3.0
...,...,...,...
464392,A3RBT0BU16G04E,B01DLRDD3Y,5.0
464393,A1G8MBCV8TUH6J,B01DLRDD3Y,5.0
464394,A2620NQRCDH4U3,B01DLRDD3Y,5.0
464395,A20WRVXD8N7QXZ,B01DLRDD3Y,3.0


Performing a merge of 'Working_dataset' and 'Product_dataset' to get all the required details together for building the Recommender system.

In [88]:
Working_dataset=pd.merge(Working_dataset,Product_dataset,on='Asin',how='inner')

Taking only the required columns and creating a pivot table with index as 'Reviewer_ID' , columns as 'Title' and values as 'Rating'.
'Model' is a pivot table.

In [89]:
Working_dataset=Working_dataset[['Reviewer_ID','Asin','Title','Rating']]
Model = Working_dataset.pivot_table(index='Reviewer_ID',columns='Title',values='Rating')

Function to find the pearson correlation between two columns or products.
Will produce result between -1 to 1.
Function will be used within the recommender function 'get_recommendations()'.

In [90]:
def pearson(x1,x2):
    x1_cor=x1-x1.mean()
    x2_cor=x2-x2.mean()
    return np.sum(x1_cor * x2_cor)/np.sqrt(np.sum(x1_cor**2) * np.sum(x2_cor**2))

Function to recommend the product based on correlation between them.
Takes 3 parameters 'Product Name', 'Model' and 'Number of Recomendations'
Will return a list in descending order of correlation and the list size depends on the input given for Number of Recomendations.

In [91]:
def get_recommendations(product_id,M,num):
    recomend=[]
    for asin in M.columns:
        if asin==product_id:
            continue
        cor=pearson(M[product_id],M[asin])
        if np.isnan(cor):
            continue
        else:
            recomend.append((asin,cor))
    recomend.sort(key=lambda tup: tup[1],reverse=True)
    return recomend[:num]

Function to replace all the html escape characters to english.

In [92]:
def escape(t):
    """HTML-escape the text in `t`."""
    return (t
        .replace("&amp;","&").replace("&lt;","<").replace("&gt;",">")
        .replace("&#39;","'").replace("&quot;",'"')
        )

Calling the recommender System by making a function call to 'get_recommendations()'.
'Perrier Lime Flavored Carbonated Mineral Water, 16.9 fl oz. Plastic Bottles (6 Count)' is the product name pass to the function i.e. if person buys 'Perrier Lime Flavored Carbonated Mineral Water, 16.9 fl oz. Plastic Bottles (6 Count)' what else can be recommended to him/her.
'Model' is passed for correlation calculation.
'5' is the maximum number of recommendation a function can return if there is some correlation.
Quantifying the correlation can be done by using correlation value given in the output.

In [94]:
rec=get_recommendations('Perrier Lime Flavored Carbonated Mineral Water, 16.9 fl oz. Plastic Bottles (6 Count)',Model,5)

Taking recommendation into DataFrame for Tabular represtation.
Takng only those values whose correlation is greater than 0.

In [95]:
Recommendation=[]
for x in rec:
    if x[1] > 0:
        Recommendation.append((escape(x[0]),x[1]))

result=pd.DataFrame(Recommendation,columns=['Product Title','Correlation '])

Taking all the recommendations into .csv file

In [97]:
result.to_csv('E:\CapstoneProject\Prime_Pantry.json\Recommendation.csv')

In [96]:
Recommendation

[('Perrier Carbonated Mineral Water, 33.8 fl oz. Plastic Bottle',
  0.6606227469540411),
 ('Perrier Green Apple Flavored Carbonated Mineral Water, 16.9 fl oz. Plastic Bottles (6 Count)',
  0.014833677226721178),
 ('Perrier Carbonated Mineral Water, 16.9 fl oz. Plastic Bottles (6 Count)',
  0.00599091439309466)]

Displaying rows of resultant DataFrame

In [98]:
result

,Product Title,Correlation
0,"Perrier Carbonated Mineral Water, 33.8 fl oz. ...",0.660623
1,Perrier Green Apple Flavored Carbonated Minera...,0.014834
2,"Perrier Carbonated Mineral Water, 16.9 fl oz. ...",0.005991
